In [ ]:
day_obs = "2024-09-01"
earliest_sim = "2023-01-01"

In [ ]:
# Validate the input
import re
assert re.match(r'^\d\d\d\d-\d\d-\d\d$', day_obs) is not None
assert re.match(r'^\d\d\d\d-\d\d-\d\d$', earliest_sim) is not None

In [ ]:
from IPython.display import display, HTML, Markdown
from collections import OrderedDict
import datetime
import sys
import os
import pandas as pd
import numpy as np
import astropy
import bokeh
import bokeh.io
import boto3
import warnings
import astropy.units as u
from astropy.time import Time, TimezoneInfo
from urllib.parse import urlparse
from lsst.resources import ResourcePath
import yaml
from contextlib import redirect_stdout
import io
from erfa import ErfaWarning

In [ ]:
os.environ['RUBIN_SIM_DATA_DIR'] = '/sdf/data/rubin/user/neilsen/data/rubin_sim_data'

In [ ]:
sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/pip_targets/lib/python3.11/site-packages')

In [ ]:
devel_versions = True
if devel_versions:
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/uranography')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-06-24/rubin_scheduler')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/rubin_sim')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-05-07/schedview')

In [ ]:
import schedview.collect.resources
import rubin_scheduler

In [ ]:
# Suppress benign warnings.

warnings.filterwarnings(
    "ignore",
    category=ErfaWarning,
    message=r".*ERFA function.*dubious year.*",
)

In [ ]:
archive_uri = "s3://rubin:rubin-scheduler-prenight/opsim/"

if urlparse(archive_uri).scheme.upper() == 'S3':
    os.environ["LSST_DISABLE_BUCKET_VALIDATION"] = "1"
    os.environ["S3_ENDPOINT_URL"] = "https://s3dfrgw.slac.stanford.edu/"

# Pre-night simulation archive overview with simulations including the `day_obs` of {{ params.day_obs }}

In [ ]:
display(Markdown(f"Including simulations inserted into the archive at `{archive_uri}` on or after {earliest_sim}."))

In [ ]:
archives_resource_path = ResourcePath(archive_uri)
num_nights = np.ceil((Time.now() - Time(earliest_sim)).to_value(format='jd')).astype(int)
sim_metadata = rubin_scheduler.sim_archive.read_archived_sim_metadata(archive_uri, num_nights=num_nights)
marked_for_deletion = []
for sim_uri in sim_metadata:
    try:
        first_day_obs = sim_metadata[sim_uri]['simulated_dates']['first_day_obs']
        last_day_obs = sim_metadata[sim_uri]['simulated_dates']['last_day_obs']
    except KeyError:
        
        try:
            first_time = sim_metadata[sim_uri]['simulated_dates']['first']
            if len(first_time) == 10:
                # If the length is 10, it's really a date.
                first_day_obs = first_time
            else:
                # day_obs is in utc-12 hours, but opsim starts in UTC
                first_day_obs = Time(first).to_datetime(timezone=TimezoneInfo(utc_offset=-12*u.hour)).date().isoformat()
            sim_metadata[sim_uri]['simulated_dates']['first_day_obs'] = first_day_obs
            
            last_time = sim_metadata[sim_uri]['simulated_dates']['last']
            if len(last_time) == 10:
                # If the length is 10, it's really a date.
                last_day_obs = last_time
            else:
                # day_obs is in utc-12 hours, but opsim starts in UTC    
                last_day_obs = Time(last).to_datetime(timezone=TimezoneInfo(utc_offset=-12*u.hour)).date().isoformat()

            sim_metadata[sim_uri]['simulated_dates']['last_day_obs'] = last_day_obs
        except KeyError:
            marked_for_deletion.append(sim_uri)
            continue
    
    if first_day_obs <= day_obs <= last_day_obs:
        this_sim_rp = ResourcePath(sim_uri)
        date_str, index_str = this_sim_rp.relative_to(archives_resource_path).split('/')
        sim_metadata[sim_uri]['sim_execution_date'] = date_str
        sim_metadata[sim_uri]['sim_index'] = int(index_str)
        sim_metadata[sim_uri]['files']['observations']['url'] = this_sim_rp.join(sim_metadata[sim_uri]['files']['observations']['name'])
        sim_metadata[sim_uri]['simulated_dates']['first_day_obs'] = first_day_obs
        
    else:
        marked_for_deletion.append(sim_uri)

for out_of_range_uri in marked_for_deletion:
    del sim_metadata[out_of_range_uri]

In [ ]:
# Sort the simulations, most recent first.
sort_by_values = np.array([(s['sim_execution_date'], s['sim_index'], s['label']) for s in sim_metadata.values()],
                         dtype=[('date', np.unicode_, 10), ('id', int), ('label', np.unicode_, 128)])
descending_indexes = np.flip(sort_by_values.argsort())
sim_metadata = OrderedDict((list(sim_metadata.keys())[i], sim_metadata[list(sim_metadata.keys())[i]]) for i in descending_indexes)

In [ ]:
sim_table_markdown = io.StringIO()
with redirect_stdout(sim_table_markdown):
    print("| Date simulation addad to archive| id | simulation |first day_obs | last day_obs | scheduler version | tags |")
    print("|---------------------------------|----|------------|--------------|--------------|-------------------|------|")
    for sim_name, metadata in sim_metadata.items():
        base_url = "https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst/schedview_notebooks/prenight/prenight"
        sim_date = metadata['sim_execution_date']
        sim_index = metadata['sim_index']
        url = f"{base_url}?day_obs={day_obs}&sim_date={sim_date}&sim_index={sim_index}"
        abbreviated_label = metadata['label'].removeprefix(f"{sim_date}/{sim_index}")
        print(f"|{sim_date}|{sim_index}", end="")
        print(f"|[{abbreviated_label}]({url})", end="")
        print(f"|{metadata['simulated_dates']['first_day_obs']}|{metadata['simulated_dates']['last_day_obs']}", end="")
        print(f"|{metadata['scheduler_version'] if 'scheduler_version' in metadata else 'unknown'}|", end="")
        if 'tags' in metadata:
            print(f"{str(metadata['tags'])}|")
        else:
            print(" |")

    print()

display(Markdown(sim_table_markdown.getvalue()))

In [ ]:
sim_metadata